In [11]:
import shutil
import subprocess
from dotenv import load_dotenv
from pydantic_ai import Agent
import yaml
from workout_builder.py.workout_definition import WorkoutDefinition
from pathlib import Path
import logging
import re
load_dotenv(override=True)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


async def generate_workout(workout_description: str, model: str, use_structured_output: bool) -> WorkoutDefinition:
    schema = WorkoutDefinition.model_json_schema()
    
    system_prompt = """You are a helpful assistant that helps translate user requests into structured workout definitions.
    Provide the workout a creative name and description. Steps should also be creatively named with appropriate descriptions and notes"""
    
    if not use_structured_output:
        system_prompt = f"""
        {system_prompt}.
        Use the json schema for the output:
        ```
        {schema}
        ```
        
        """
        agent = Agent(model=model)
        prompt = f"""Help me generate a json file for the workout: {workout_description}"""
        result = await agent.run(system_prompt + prompt)

        # extract text between markdown code blocks
        match = re.search(r"```(json)?\n(.*?)\n```", result.output, re.DOTALL)
        if match:
            json_text = match.group(2)
            result.output = json_text
        else:
            print("No code block found")

        try:
            workout: WorkoutDefinition = WorkoutDefinition.model_validate_json(json_text)
        except:
            print("Failed to parse JSON, falling back to structured output")
            print(json_text)
    else:
        agent = Agent(model=model, system_prompt=system_prompt)
        prompt = f"""Help me generate a workout for: {workout_description}"""
        result = agent.run_sync(prompt,output_type=WorkoutDefinition)

        workout = result.output

    print(f"💡 Created workout: {workout.metadata.name}")
    return workout


def _sanitize_name(n: str) -> str:
    # Mirror Java sanitizeName: keep A-Za-z0-9 _- then trim length 30 then replace spaces with '_' for filename
    cleaned = re.sub(r"[^A-Za-z0-9 _-]", "", n)[:30]
    return cleaned.replace(" ", "_")

def encode_to_fit(yaml_file: str, fit_file: str):
    """Encode a YAML workout to FIT using Java encoder without passing an explicit name.
    The Java program will read metadata.name. We post-compute expected filename and move it to fit_file."""
    JAVA_DIR = Path(
        "/Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/java"
    )
    JAVA_BUILD = JAVA_DIR / "build"
    JAVA_LIB_DIR = JAVA_DIR / "lib"
    FIT_JAR = JAVA_LIB_DIR / "fit.jar"
    SNAKEYAML_JAR = JAVA_LIB_DIR / "snakeyaml-2.2.jar"
    CLASSPATH = ":".join([str(JAVA_BUILD), str(FIT_JAR), str(SNAKEYAML_JAR)])
    ENCODER_CLASS = "com.neuraltag.workout.EncodeYamlWorkout"

    # Parse YAML to discover metadata.name for expected filename
    with open(yaml_file, "r") as yf:
        data = yaml.safe_load(yf)
    meta_name = data.get("metadata", {}).get("name", "Workout")
    produced_filename = f"{_sanitize_name(meta_name)}.fit"

    command = f"java -cp {CLASSPATH} {ENCODER_CLASS} {yaml_file}"
    subprocess.run(command, shell=True, check=True)

    # Move generated FIT to desired location
    shutil.move(produced_filename, fit_file)
    logger.info(f"Encoded workout using metadata name '{meta_name}' -> {fit_file}")



In [13]:

logger.info("Starting workout generation")
# input = "5 min warmup + 5 x (30s@4:20 + 30s rest) + 6x[1km@(3:45-3:50) + 1 min rest) ] + 15 min cool down @ 6:00"
input = """
10min easy warm up at 5:30-5:52/km
5x [30sec builds to planned session pace; 30 sec easy],
followed by some coordination drills

7x1km in 3:54-4:07
1min REST 

5min easy cool down at 5:30-5:52/km
"""

logger.info(f"Converting `{input}` to yaml")
workout = await generate_workout(
    workout_description=input,
    # model="google-gla:gemma-3-27b-it",
    model = "google-gla:gemini-2.5-flash-lite",
    use_structured_output=False,
)

print(yaml.dump(workout.model_dump(mode="json", exclude_none=True),sort_keys=False))


INFO:__main__:Starting workout generation
INFO:__main__:Converting `
10min easy warm up at 5:30-5:52/km
5x [30sec builds to planned session pace; 30 sec easy],
followed by some coordination drills

7x1km in 3:54-4:07
1min REST 

5min easy cool down at 5:30-5:52/km
` to yaml
INFO:__main__:Converting `
10min easy warm up at 5:30-5:52/km
5x [30sec builds to planned session pace; 30 sec easy],
followed by some coordination drills

7x1km in 3:54-4:07
1min REST 

5min easy cool down at 5:30-5:52/km
` to yaml
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


💡 Created workout: The Rocket Builder
version: 1
metadata:
  name: The Rocket Builder
  description: This workout is designed to build your engine for longer efforts and
    prepare you for faster paces. It starts with a steady warm-up, progresses through
    short, sharp intervals with recovery, hits key race pace segments, and finishes
    with a solid cool-down.
  sport: running
options:
  hr_offset_mode: add_100
  default_intensity: active
  repeat_mode_default: controller
steps:
- name: Gentle Ignition
  intensity: warmup
  type: simple
  duration:
    value: 10.0
    unit: minutes
    open: false
  target:
    type: pace_range
    low: '5:52'
    high: '5:30'
    unit: min_per_km
  note: Start with a relaxed pace, focusing on smooth, easy breathing and a comfortable
    stride. Let your body gradually wake up.
- name: Ignition Sequence
  type: group
  repeat: 5
  children:
  - name: Burner Boost
    intensity: active
    type: simple
    duration:
      value: 30.0
      unit: se

In [14]:
name = workout.metadata.name.replace(" ", "_").replace("-", "_").lower()
yaml_file = f"/Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/{name}.yaml"
fit_file = f"/Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/{name}.fit"
with open(yaml_file, "w") as f:
    yaml.dump(workout.model_dump(mode="json", exclude_none=True), f,sort_keys=False,indent=4)

# Encode without passing name (Java reads metadata.name)
encode_to_fit(yaml_file, fit_file)
logger.info(f"💾 Wrote workout to {yaml_file}\nand {fit_file}")


INFO:__main__:Encoded workout using metadata name 'The Rocket Builder' -> /Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/the_rocket_builder.fit
INFO:__main__:💾 Wrote workout to /Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/the_rocket_builder.yaml
and /Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/the_rocket_builder.fit
INFO:__main__:💾 Wrote workout to /Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/the_rocket_builder.yaml
and /Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/the_rocket_builder.fit


Wrote The_Rocket_Builder.fit
Created FIT workout with 11 steps (including repeat controllers).


In [20]:
pd.DataFrame(workout.expand())

,index,kind,name,intensity,duration,target,note,repeat_from,additional_times
0,0,simple,Gentle Ignition,warmup,"{'type': 'time', 'time_ms': 600000}","{'kind': 'pace_range', 'low': 2841, 'high': 3030}","Start with a relaxed pace, focusing on smooth,...",NaN,NaN
1,1,simple,Burner Boost,active,"{'type': 'time', 'time_ms': 30000}","{'kind': 'pace', 'low': 4167, 'high': 4167}",Gradually increase your pace to the target pac...,NaN,NaN
2,2,simple,Controlled Descent,rest,"{'type': 'time', 'time_ms': 30000}","{'kind': 'pace_range', 'low': 2841, 'high': 3030}","Ease back to a relaxed, easy pace. Allow your ...",NaN,NaN
3,3,repeat_controller,NaN,NaN,NaN,NaN,NaN,1.0,4.0
4,4,simple,High Knees,active,"{'type': 'time', 'time_ms': 20000}",{'kind': 'open'},Focus on bringing your knees up high and drivi...,NaN,NaN
5,5,simple,Butt Kicks,active,"{'type': 'time', 'time_ms': 20000}",{'kind': 'open'},Bring your heels up towards your glutes. This ...,NaN,NaN
6,6,simple,High Knee & Butt Kick Combo,active,"{'type': 'time', 'time_ms': 20000}",{'kind': 'open'},Alternating between high knees and butt kicks ...,NaN,NaN
7,7,simple,Target Kilometer,active,"{'type': 'distance', 'distance_cm': 100000}","{'kind': 'pace_range', 'low': 4049, 'high': 4274}",Hold this challenging but sustainable pace. Fo...,NaN,NaN
8,8,simple,Recovery Interval,rest,"{'type': 'time', 'time_ms': 60000}",{'kind': 'open'},Walk or jog very slowly. Allow your body to re...,NaN,NaN
9,9,repeat_controller,NaN,NaN,NaN,NaN,NaN,7.0,6.0
